In [58]:
import pandas as pd
import os

# Get data for data discovery

In [59]:
folder_processed_data_path = './processed_data/'
processed_website_data_folder = './processed_data/website/'

We import the dataframe computed in preprocessing.

In [60]:
# import the name by movie
print("name_by_movie_ordered_pvalue_10_5_df :")
name_by_movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'name_by_movie_ordered_pvalue_10_5_df.csv'))
# name_by_movie_df.set_index(['wiki_ID', 'char_words', 'gender'], inplace=True)
display(name_by_movie_df.sample(2))
print(f"length : {len(name_by_movie_df)}\n")

# import movie genres
print("movie_genres_df :")
movie_genres_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_genres_df.csv'))
display(movie_genres_df.sample(2))
print(f"length : {len(movie_genres_df)}\n")

# import the movie dataframe
print("movie_df :")
movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_df.csv'))
# movie_df.set_index(['wiki_ID'], inplace=True)
display(movie_df.sample(2))
print(f"length : {len(movie_df)}")

name_by_movie_ordered_pvalue_10_5_df :


,wiki_ID,char_words,order,gender,t_stat,p_value,slope_change
136127,23535297,Milo,0.0,M,-1.587552,0.140695,0.001719
122820,20581179,Seeta,3.0,F,0.000000,1.000000,0.000000


length : 172906

movie_genres_df :


,wiki_ID,genre
79192,6484887,Drama
71263,5557065,Science Fiction


length : 213854

movie_df :


,wiki_ID,mov_name,year,month,revenue,numVotes,averageRating,poster_url,IMDB_ID
14445,13079614,The Girl in the Limousine,1924,7.0,NaN,19,6.3,NaN,tt0014946
15337,21711315,Don't Panic Chaps!,1959,NaN,NaN,169,5.6,NaN,tt0052750


length : 65503


In [61]:
# keep only the movie_id and name and drop duplicates as we remove the gender column
char_name_per_movie = name_by_movie_df[['wiki_ID', 'char_words']].drop_duplicates()
display(char_name_per_movie.sample(2))
print(f"Length change : {len(name_by_movie_df)} -> {len(char_name_per_movie)}")

,wiki_ID,char_words
18531,644385,Young
26272,1036025,Jin


Length change : 172906 -> 167984


In [62]:
# add a column with the genre of the movie
char_name_per_movie_with_genre = char_name_per_movie.merge(movie_genres_df, on='wiki_ID', how='left').copy(deep=True)

# rearrange and rename the columns
char_name_per_movie_with_genre = char_name_per_movie_with_genre[['genre', 'char_words', 'wiki_ID']].copy(deep=True)
char_name_per_movie_with_genre.rename(columns={'char_words': 'char_name', 'wiki_ID': 'movie_id'}, inplace=True)

display(char_name_per_movie_with_genre.sample(2))
print(f"Length change : {len(char_name_per_movie)} -> {len(char_name_per_movie_with_genre)}") # the increase is due to the fact that some movies have multiple genres

,genre,char_name,movie_id
690517,Crime Fiction,Jody,28222345
148909,Cult,Robin,967721


Length change : 167984 -> 739925


In [63]:
# for each name and genre, count the number of movies
movie_count_per_name_and_genre = char_name_per_movie_with_genre.groupby(['genre', 'char_name']).count().reset_index()
movie_count_per_name_and_genre.rename(columns={'movie_id': 'nb_movies'}, inplace=True)
display(movie_count_per_name_and_genre.sample(10))
print(f"Length of movie_count_per_name_and_genre : {len(movie_count_per_name_and_genre)}")

,genre,char_name,nb_movies
241454,Thriller,Tobias,3
214817,Short Film,Langston,1
50418,Chinese Movies,Riko,1
250500,World cinema,Dhiren,1
47497,Children's Fantasy,Margaret,1
88805,Culture & Society,Captain,1
89980,Detective,Lorenzo,1
114853,Family Drama,Willa,1
25940,Anime,Arnie,1
228598,Suspense,Cookie,1


Length of movie_count_per_name_and_genre : 254789


In [64]:
# drop the non relevant name 'The' from the dataframe
movie_count_per_name_and_genre = movie_count_per_name_and_genre[movie_count_per_name_and_genre['char_name'] != 'The'].copy(deep=True)

In [65]:
# For each genre, keep only the 10 names with the most counts
movie_count_per_name_and_genre.reset_index(inplace=True)
movie_count_per_name_and_genre.sort_values(by=['genre', 'nb_movies'], ascending=False, inplace=True)
top10_per_genre = movie_count_per_name_and_genre.groupby('genre').head(10).reset_index(drop=True)
top10_per_genre.set_index(['genre', 'char_name'], inplace=True)
display(top10_per_genre)

index  nb_movies
genre       char_name                   
Zombie Film Alice      254325          6
            Frank      254486          5
            Sheriff    254731          5
            Ben        254358          4
            Doctor     254447          4
...                       ...        ...
Absurdism   Doctor        123          4
            George        176          4
            King          271          4
            Man           304          4
            Roger         414          4

[3157 rows x 2 columns]

In [68]:
# keep only the rows with a genre among the 5 genres with the highest number of names
top5_genres = movie_count_per_name_and_genre.groupby('genre').sum().sort_values(by='nb_movies', ascending=False).head(5).index
top10_name_top5_genre = top10_per_genre.loc[top5_genres, :].copy(deep=True)
top10_name_top5_genre.drop(columns=['index'], inplace=True)
print(top10_name_top5_genre)

                        nb_movies
genre        char_name           
Drama        John             701
             Jack             426
             Man              397
             Mary             379
             Joe              338
             Paul             334
             Frank            333
             George           325
             David            324
             Tom              307
Comedy       John             261
             Girl             250
             Jack             249
             Man              234
             Joe              193
             Frank            187
             George           186
             Sam              181
             Mary             180
             Tom              165
Thriller     John             304
             Jack             246
             Man              185
             Frank            183
             Agent            175
             Captain          155
             James            152
             D

In [71]:
# rename the genre 'Romance Film' to 'Romance'
top10_name_top5_genre.reset_index(inplace=True)
top10_name_top5_genre.loc[top10_name_top5_genre['genre'] == 'Romance Film', 'genre'] = 'Romance'
top10_name_top5_genre.set_index(['genre', 'char_name'], inplace=True)
display(top10_name_top5_genre.sample(2))

,,nb_movies
genre,char_name,
Action,Joe,105
Romance,George,125


# Export the dataframes

In [72]:
# Export top10_name_top5_genre.csv
display(top10_name_top5_genre.reset_index().sample(2))
top10_name_top5_genre.to_csv(os.path.join(processed_website_data_folder, 'top10_name_top5_genre.csv'), index=True)

,genre,char_name,nb_movies
8,Drama,David,324
41,Action,Captain,210
